In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import scipy.optimize as opt
nms = np.loadtxt('NewMacroSeries.txt', delimiter = ',')
ct = nms[:,0]
kt = nms[:,1]
wt = nms[:,2]
rt = nms[:,3]
yt = nms[:,4]

# ct, kt, wt, rt, yt = dt[:,0], dt[:,1], dt[:,2], dt[:,3], dt[:,4]

In [2]:
k_1 = np.mean(kt)
S = 1000
T = 100
# Random draw from Uniform(0,1)
U = np.random.uniform(0,1,(T,S))
beta = 0.99

In [3]:
def SIM(alpha, beta, rho, mu, sigma, U):
    eps = sts.norm.ppf(U,0,sigma)
    z = np.zeros((T,S))
    z_t_1 = mu
    for i in range(T):
        z_t = rho*z_t_1 + (1-rho)*mu + eps[i,:]
        z[i,:] = z_t
        z_t_1 = z_t
        
    k = np.zeros((T,S))
    k_lag = np.zeros((T,S))
    k_t_1 = k_1
    for i in range(T):
        k_t = alpha*beta*np.exp(z[i,:])*k_t_1**alpha
        k_lag[i,:] = k_t_1
        k[i,:] = k_t
        k_t_1 = k_t
        
    w = (1-alpha)*np.exp(z)*k_lag**alpha
    r = alpha*np.exp(z)*k_lag**(alpha-1)
    c = w + r*k_lag - k
    y = np.exp(z)*k_lag**alpha
    return c,k_lag,k,w,r,y

In [4]:
def model_mom(alpha, beta, rho, mu, sigma,U):
    c,k_lag,k,w,r,y = SIM(alpha, beta, rho, mu, sigma, U)
    
    m1 = np.mean(c)
    m2 = np.mean(k)
    m3 = np.mean(c/y)
    m4 = np.mean(np.std(y,axis=0))
    corr_c = np.zeros((1,S))
    corr_ck = np.zeros((1,S))
    for i in range(S):
        corr_c[0,i] = np.corrcoef(c[0:99,i],c[1:100,i])[0,1]
        corr_ck[0,i] = np.corrcoef(c[:,i],k_lag[:,i])[0,1]
        
    m5 = np.mean(corr_c)
    m6 = np.mean(corr_ck)
    model_mom = np.array([m1,m2,m3,m4,m5,m6])
    return model_mom

In [5]:
def err_vec(alpha, beta, rho, mu, sigma, U):
    
    data_moms = np.array([np.mean(ct),np.mean(kt),np.mean(ct/yt),np.var(yt),
                         np.corrcoef(ct[0:99],ct[1:100])[0,1],
                         np.corrcoef(ct,kt)[0,1]])
    
    model_moms = model_mom(alpha, beta, rho, mu, sigma, U)
    
    return (model_moms - data_moms)/data_moms

def crit(params, *args):
    alpha, rho, mu, sigma = params
    beta, U = args
    err = err_vec(alpha, beta, rho, mu, sigma, U)
    W = np.eye(6)
    return err @ W @ err.T

In [6]:
params_init = np.array([0.5, 0.8, 11, 1])
bounds = ((1e-10,1-1e-10),(-1+1e-10,1-1e-10),(5,14),(1e-10,None))

result = opt.minimize(crit, params_init, args = (beta,U), method = 'L-BFGS-B', bounds = bounds,
                     options = {'eps':1e-06})
alpha_smm, rho_smm, mu_smm, sigma_smm = result.x
print(result)

      fun: 1.0000065721505345
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -5.89678750e-04,  -2.90101276e-04,  -9.90858506e-05,
        -7.11875003e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 295
      nit: 38
   status: 0
  success: True
        x: array([ 0.42101743,  0.96089798,  9.61472007,  0.19394674])


In [8]:
data_moms = np.array([np.mean(ct),np.mean(kt),np.mean(ct/yt),np.var(yt),
                      np.corrcoef(ct[0:99],ct[1:100])[0,1],np.corrcoef(ct,kt)[0,1]])

In [9]:
print('alpha SMM = ', alpha_smm)
print('rho SMM = ', rho_smm)
print('mu SMM = ', mu_smm)
print('sigma SMM = ', sigma_smm)
print('the vector of moment differences at the optimum =', 
      err_vec(alpha_smm, beta, rho_smm, mu_smm, sigma_smm, U)*data_moms)
print('the criterion function value =', result.fun)

alpha SMM =  0.421017428155
rho SMM =  0.960897979397
mu SMM =  9.61472007024
sigma SMM =  0.19394674165
the vector of moment differences at the optimum = [  7.12708664e+03  -5.20533439e+03  -1.00725387e-03  -2.83778107e+13
   1.08211851e-03  -1.35880673e-03]
the criterion function value = 1.00000657215
